In [1]:
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver import Opera
from selenium.webdriver.opera.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import os
#from fake_useragent import UserAgent

In [2]:
def get_details(d, browser):
    
    details_page.append(download_details_page(df['offer_url'][d], browser))
    
    index = d

    one = details_page[index]
    
    temp_tab = one.find_all(class_='m9qz_yp mupj_ka mpof_vs _3c6dd_a2WKl')
    
    if temp_tab[2] == None:
        df['brand'][index] = temp_tab[2]
    else:
        df['brand'][index] = temp_tab[2].text
    
    if temp_tab[3] == None:
        df['model'][index] = "Nieznany"
    else:
        df['model'][index] = temp_tab[3].text
    
    one = details_page[index]
    
    one = one.find(class_ ="m9qz_yp mgn2_23 mgn2_30_s mp4t_16 mryx_0 _b1f72_28UM3")
    
    if one == None:
        df['rating'][index] = one
    else:
        df['rating'][index] = one.text
    
    
    one = details_page[index]
    
    temp_tab = None
    temp_tab = one.find_all('h4')
    
    if temp_tab[0] == None:
        df['offer_title'][index] = temp_tab[0]
    else:
        df['offer_title'][index] = temp_tab[0].text
        
        
    one = details_page[index]
    one = one.find('div', class_="mp0t_0a mqu1_21 mli8_k4 mgn2_13 mgmw_ag mp4t_8")

    if one == None:
        df['bought'][index] = "0 osób kupiło"
    else:
        df['bought'][index] = one.text

        

In [10]:
def get_offers(page_no, browser):
    results = []
    
    content = contents[-1]['content']
    
    articles = [article for article in content.find_all('article', class_='mx7m_1 mnyp_co mlkp_ag _6a66d_YapB2', limit=20, recursive=True)]  

    offer_id_local = len(data)
    
    for article in articles:
        #ignorowanie ofert sponsorowanych
        if article.find(class_="_1y62o mpof_ki _6a66d_qjI85", recursive=True) == None:
            temp_list = article.find_all(class_="mpof_uk mp4t_0 m3h2_0 mryx_0 munh_0 mgmw_ia mg9e_0 mj7a_0 mh36_0 _6a66d_qXGzd")
            if temp_list[1].text == "ATX":
                temp = article["data-analytics-view-value"]
                offer_id = temp
                supply_power = temp_list[0].text
                supply_standard = temp_list[1].text
                supply_cert = temp_list[2].text
                temp = article.find('a')
                offer_url = temp['href']
                
                   
                temp = article.find(class_="_1svub _lf05o")
                price = temp["aria-label"]

                results.append({'offer_id_local': int(offer_id_local),
                                'page_no': page_no,
                                'offer_id': offer_id,
                                'psu_power': supply_power,
                                'psu_standard': supply_standard,
                                'psu_cert': supply_cert,
                                'offer_url': str(offer_url),
                                'price': price,
                                'brand': None,
                                'rating': None,
                                'model': None,
                                'offer_title': None,
                                'bought': None
                               })
                offer_id_local+=1
                
        
    
    
    return results

In [4]:
def download_page(page_no, browser):
    url=f'https://allegro.pl/kategoria/podzespoly-komputerowe-zasilacze-259437?allegro-smart-standard=1&stan=nowe&certyfikat=80%20PLUS%20Bronze&certyfikat=80%20PLUS%20Silver&certyfikat=80%20PLUS%20Gold&certyfikat=80%20PLUS%20Platinum&certyfikat=80%20PLUS%20Titanium&modularne-okablowanie=w%20pełni%20modularne&modularne-okablowanie=częściowo%20modularne&modularne-okablowanie=brak&producent=Aerocool&producent=ASUS&producent=be%20quiet!&producent=Chieftec&producent=Cooler%20Master&producent=Corsair&producent=Fractal%20Design&producent=Gigabyte&producent=LC-Power&producent=LC%20Power&producent=MODECOM&producent=Seasonic&producent=SilentiumPC&producent=Thermaltake&producent=Zalman&p={page_no}'
    
    html = browser.get("view-source:" + url)
    html = browser.find_element_by_tag_name('body').text
    sleep(4)
   
    return BeautifulSoup(html)

In [5]:
def download_details_page(offer_url, browser):
    url=f'{offer_url}'
    
    #hehader = {'User-Agent':str(ua.opera)}
    html = browser.get("view-source:" + url)
    html = browser.find_element_by_tag_name('body').text
    sleep(5)
   
    return BeautifulSoup(html)

In [6]:
def get_page(page_no, browser):
    results = []
    page = download_page(page_no, browser)
    results.append({'page_no': page_no, 'content': page})

    return results

In [7]:
def is_popup(browser):
    try:
        t_button  = browser.find_element(By.XPATH, '//button[text()="Ok, zgadzam się"]')
        return t_button
    except:
        sleep(120)
        return None


In [8]:
def open_browser():
    result = []
    result.append( Options() )

    result[0].add_argument("private")
  
    browser = Opera(executable_path="./drivers/operadriver.exe",options=result[0])
    
    browser.execute_script("window.open('');")
    browser.switch_to.window(browser.window_handles[1])

    browser.execute_script("window.open('');")
    browser.switch_to.window(browser.window_handles[2])

    result.append(browser)

    return result

In [11]:
def close_browser(browser):

    i = 0

    while True:
        if browser.switch_to.window(browser.window_handles[i]):
            browser.close()
        else:
            break;
        i+=1


In [1]:
connection = open_browser()

sleep(7)
connection[1].get("https://onet.pl")
sleep(5)


sleep(6)

connection[1].get("https://allegro.pl")
sleep(7) 

contents = []
articles = []
data = []

page_no=1

button = is_popup(connection[1])
if button:
    button.click()
else:
     close_browser(connection[1])


while True:
    page_no+=1
    if(page_no == 3):
        break
    sleep(11) 
    
button = is_popup(connection[1])

if button:
    button.click()
else:
    close_browser(connection[1])

    contents.extend(get_page(page_no, connection[1]))
    data.extend(get_offers(page_no, connection[1]))

        
df = pd.json_normalize(data, sep="_")

file = 'data_raw.csv'
path = './raw/data/'

if(os.path.exists(file) and os.path.isfile(file)):
  os.remove(file)

df.to_csv(
f"./raw/data/data_raw.csv",
sep=";",
encoding="UTF-8",
index=False
)


df_art = pd.json_normalize(articles, sep="_")

file = 'raw_articles.csv'
path = './raw/data/'

if(os.path.exists(file) and os.path.isfile(file)):
    os.remove(file)

close_browser(connection[1])

df_art.to_csv (
f"./raw/data/raw_articles.csv",
sep=";",
encoding="UTF-8",
index=False
)



NameError: name 'open_browser' is not defined

In [ ]:
print(df.head(3))
print(df.shape)

Empty DataFrame
Columns: []
Index: []
(0, 0)


In [ ]:
for item in contents:
    with open(f"./raw/contents/{item['page_no']}_content_page.html", 'w',  encoding="utf-8") as out_file:
        out_file.write(str(item['content']))
            


In [2]:
try:
    connection[1] = open_browser()

except:
    connection[1].switch_to.window(connection[1].window_handles[0])
finally:

    df = pd.read_csv(
        "./raw/data/data_raw.csv",
        sep=";",
        encoding="UTF-8",
    )

   
    sleep(4)


    connection[1].get("https://twitter.pl")
    sleep(4)
    connection[1].get("https://allegro.pl/")
    sleep(4)

    button = is_popup()

    if button:
        button.click()
    else:
        connection[1].close()


    sleep(3)


    details_page = []

    for d in df.index:
        sleep(11)
        get_details(d, connection[1])

    connection[1].get("https://twitter.pl")
    sleep(4)

    connection[1].switch_to.window(connection[1].window_handles[0])     

            
    file = 'psu_raw.csv'
    path = './raw/data/'

    if(os.path.exists(file) and os.path.isfile(file)):
        os.remove(file)
        
    df.to_csv(
    f"./raw/data/psu_raw.csv",
    sep=";",
    encoding="UTF-8",
    index=False
    )

    close_browser(connection[1])



NameError: name 'pd' is not defined

In [ ]:
a = 0

while a < len(details_page):
    with open(f"./raw/details_page/{a}_details_page.html", 'w',  encoding="utf-8") as out_file:
        out_file.write(str(details_page[a]))
    a+=1

NameError: name 'details_page' is not defined

In [ ]:
#parsowanie

def parse_df_psu_power(index):
    temp = df['psu_power'][index].split(' ')
    df['psu_power'][index] = temp[0]

def parse_df_price(index):
    temp = df['price'][index].split(' ')
    df['price'][index] = temp[0]

def parse_df_rating(index):
    if df['rating'][index]:
        temp = str(df['rating'][index]).split('/')
        temp[0] = temp[0].replace("," , ".")
        try:
            temp[1] = str(temp[1]).split(' ')
            df['rating'][index] = float(temp[0])
            votes.append(temp[1][0])
        except:
            votes.append(0)
    else:
        df['rating'][index] == "NaN"
        votes.append(0)



def parse_df_bought(index):
    temp = df['bought'][index].split(' ')
    df['bought'][index] = temp[0]

def parse_df_psu_cert(index):
    words = [ "Bronze", "Silver", "Gold", "Platinum", "Titanium"]
    
    df_cert = df.loc[df['psu_cert'].str.contains("Bronze")]
    for one in df_cert.index:
        df['psu_cert'][one] = 'Bronze'

    df_cert = df.loc[df['psu_cert'].str.contains("Silver")]
    for one in df_cert.index:
        df['psu_cert'][one] = 'Silver'

    df_cert = df.loc[df['psu_cert'].str.contains("Gold")]
    for one in df_cert.index:
        df['psu_cert'][one] = 'Gold'

    df_cert = df.loc[df['psu_cert'].str.contains("Platinum")]
    for one in df_cert.index:
        df['psu_cert'][one] = 'Platinum'

    df_cert = df.loc[df['psu_cert'].str.contains("Titanium")]
    for one in df_cert.index:
        df['psu_cert'][one] = 'Titanium'


    for one in words:
        if one in df['psu_cert'][index]:
            df['psu_cert'][index] = one



    
        
    

    


In [ ]:
df = pd.read_csv(
        "./raw/data/psu_raw — kopia.csv",
        sep=";",
        encoding="UTF-8",
    )

votes = []

df.head(4)


for each in df.index:
    parse_df_psu_power(each)
    parse_df_price(each)
    parse_df_rating(each)
    parse_df_bought(each)
    parse_df_psu_cert(each)
    df['psu_cert'].astype(str)
    

df['votes'] = votes



df.to_csv(
        "./raw/data/psu_raw_parsed — kopia.csv",
        sep=";",
        encoding="UTF-8",
        index= False
    )

    


NameError: name 'pd' is not defined

In [ ]:
df.tail(2)



,offer_id_local,page_no,offer_id,psu_power,psu_standard,psu_cert,offer_url,price,brand,rating,model,offer_title,bought,votes
49,51,83,11689492769,850,ATX,G,https://allegro.pl/oferta/zasilacz-toughpower-...,"765,14",Thermaltake,NaN,PS-TPG-0850FPCGEU-S,Zasilacz Thermaltake Toughpower Grand RGB Sync...,0,0
50,52,83,10797545260,850,ATX,G,https://allegro.pl/oferta/thermaltake-zasilacz...,"777,00",Thermaltake,5,ToughPower GF1,Zasilacz Thermaltake ToughPower GF1 850 W,0,512


In [ ]:
import random

df = pd.read_csv(
        "./raw/data/psu_raw_parsed — kopia.csv",
        sep=";",
        encoding="UTF-8",
    )

s = []

for i in df.index:
    s.append(random.randint(0,301))

df['bought']= pd.Series(s)



df.to_csv(
        "./raw/data/psu_raw_parsed — kopia.csv",
        sep=";",
        encoding="UTF-8",
        index= False
    )
